In [ ]:
import abc
import matplotlib.pyplot as plt

class HilbertSFC(object):
	__metaclass__ = abc.ABCMeta

	def __init__(self, x, y, turtle_x, turtle_y, stepsize, spacetree):
		self.x = x
		self.y = y
		self.turtle_x = turtle_x
		self.turtle_y = turtle_y
		self.stepsize = stepsize
		self.spacetree = spacetree
		self.stptr = -1

	# function to 'expand' a specified leaf of a spacetree into a refined node:
	def adapt_tree(self, position):
		# spacetree: spacetree in bitstream notation
		# position: index of the node ot expand

		# do nothing for an interior node
		if(self.spacetree[position] != 1):
			# replace a 0 bit at given position by a uniform tree of depth 1
			self.spacetree[:position] + [1,0,0,0,0] + self.spacetree[position+1:]

	# helper functions for plotter graphics

	def up(self):
		self.turtle_y += self.stepsize

	def down(self):
		self.turtle_y -= self.stepsize

	def left(self):
		self.turtle_x -= self.stepsize

	def right(self):
		self.turtle_x += self.stepsize

	# step down in the spacetree hierarchy
	# (from the center of the partent to the center of the child)
	def fine(self, x_shift, y_shift):
		self.stepsize = self.stepsize/2
		self.turtle_x += x_shift*self.stepsize/2
		self.turtle_y += y_shift*self.stepsize/2

	# step up in the spacetree hierarchy
	# (from the center of the child to the center of the parent)
	def coarse(self, x_shift, y_shift):
		self.turtle_x += x_shift*self.stepsize/2
		self.turtle_y += y_shift*self.stepsize/2
		self.stepsize = self.stepsize*2

	# mark the current position of the turtle
	def mark(self):
		self.x.append(self.turtle_x)
		self.y.append(self.turtle_y)

	# algorithms for the adaptive Hilbert curve
	def H(self):

		self.stptr += 1

		if self.spacetree[self.stptr] == 0 :
			self.mark()
		else:
			# recursive calls to children
			self.fine(-1, -1)
			self.A()
			self.up()
			self.H()
			self.right()
			self.H()
			self.down()
			self.B()
			self.coarse(-1,1)

	def A(self):

		self.stptr += 1

		if self.spacetree[self.stptr] == 0 :
			self.mark()
		else:
			# recursive calls to children
			self.fine(-1, -1)
			self.H()
			self.right()
			self.A()
			self.up()
			self.A()
			self.left()
			self.C()
			self.coarse(1,-1)

	def B(self):

		self.stptr += 1

		if self.spacetree[self.stptr] == 0 :
			self.mark()
		else:
			# recursive calls to children
			self.fine(1, 1)
			self.C()
			self.left()
			self.B()
			self.down()
			self.B()
			self.right()
			self.H()
			self.coarse(-1,1)

	def C(self):

		self.stptr += 1

		if self.spacetree[self.stptr] == 0 :
			self.mark()
		else:
			# recursive calls to children
			self.fine(1, 1)
			self.B()
			self.down()
			self.C()
			self.left()
			self.C()
			self.up()
			self.A()
			self.coarse(1,-1)

#-----------------------------------------------------------------------------------------------
# function to construct a uniformly refined tree of a given length
def gen_full_tree(depth):
	# depth: depth of the generated spacetree
	if depth == 0:
		spacetree = [0]
	else:
		spacetree = [1]
		for k in range(0,4):
			spacetree += gen_full_tree(depth-1)
	return spacetree

# "Fibonnacci-SpaceTree" (the depth decreases for the two last subtrees):
def gen_fib_tree(depth):
	# depth: depth of the generated spacetree
	if depth == 0:
		spacetree = [0]
	elif depth == 1:
		spacetree = [1]
		for k in range(0,4):
			spacetree += gen_fib_tree(depth-1)
	elif depth == 2:
		spacetree = [1]
		for k in range(0,2):
			spacetree += gen_fib_tree(depth-1)
		for k in range(2,4):
			spacetree += gen_fib_tree(depth-2)
	else:
		spacetree = [1] + gen_fib_tree(depth-1)
		for k in range(1,3):
			spacetree += gen_fib_tree(depth-2)
		spacetree += gen_fib_tree(depth-3)
	return spacetree

# functions to plot SFC

def plotCurveDelayed(x, y, delay):
	px = []
	py = []
	fig, ax = plt.subplots()
	for i in xrange (len(x)):
		px.append(x[i])
		py.append(y[i])
		if i == 0:
			points, = ax.plot(px, py)
			ax.set_xlim(min(x)-0.05, max(x)+0.05)
			ax.set_ylim(min(y)-0.05, max(y)+0.05)
		else:
			points.set_data(px, py)
		plt.pause(delay)
		

def plotLineStrip(x, y, title = None):
	fig, ax = plt.subplots()
	plt.plot(x, y, linewidth = 2.0, color = 'g')
	if title is not None:
		plt.title(title)
	ax.set_xlim(min(x)-0.05, max(x)+0.05)
	ax.set_ylim(min(y)-0.05, max(y)+0.05)

def plotLineStripPartition(part_x, part_y, colors, title = None):
	fig, ax = plt.subplots()
	plt.hold(True)
	for i in range(0,len(part_x)):
		plt.plot(part_x[i], part_y[i], linewidth = 2.0, color = colors[i])
	if title is not None:
		plt.title(title)
	ax.set_xlim(min(min(part_x))-0.05, max(max(part_x))+0.05)
	ax.set_ylim(min(min(part_y))-0.05, max(max(part_y))+0.05)
	plt.hold(False)		    

# partition splits the given list pts into number partitions (each partition is again a list)
def partition(x, y, num):
	parts_x = [x[(num-1)*(len(x)/num):]]
	parts_y = [y[(num-1)*(len(y)/num):]]
	
	for i in range(num-1,1,-1):
		parts_x = [x[(i-1)*(len(x)/num):i*(len(x)/num)]] + parts_x
		parts_y = [y[(i-1)*(len(y)/num):i*(len(y)/num)]] + parts_y
		
	parts_x = [x[:(len(x)/num)]] + parts_x
	parts_y = [y[:(len(y)/num)]] + parts_y

	return (parts_x, parts_y)


###############################################################
# This is the main function of this python script.
###############################################################
if __name__ == '__main__':

	# set delay for animation
	delay = 0.0

	print 'full  spacetree:' + str(gen_full_tree(2))
	print 'fib   spacetree:' + str(gen_fib_tree(3))

	hsfc1 = HilbertSFC([], [], 0.5, 0.5, 1.0, \
			 [1,0,0,0,1,0,1,0,0,0,0,0,0])
	hsfc1.H()

	if delay > 0.0:
		plotCurveDelayed(hsfc1.x, hsfc1.y, delay)
	else:
		plotLineStrip(hsfc1.x, hsfc1.y, None)

	hsfcfib9 = HilbertSFC([], [], 0.5, 0.5, 1.0, gen_fib_tree(9))
	hsfcfib9.H()

	if delay > 0.0:
		plotCurveDelayed(hsfcfib9.x, hsfcfib9.y, delay)
	else:
		plotLineStrip(hsfcfib9.x, hsfcfib9.y, None)

	colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

	part_x, part_y = partition(hsfcfib9.x, hsfcfib9.y, 7)

	plotLineStripPartition(part_x, part_y, colors)

	plt.show()